In [94]:
'''
Routine to plot results with pareto-optimal supermarkets of a query
'''
#%matplotlib notebook
%matplotlib qt

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import numpy as np

#for drawing the pareto front
import matplotlib.tri as mtri

#for mouseover effects
import mplcursors


'''
Pareto Source Code from: 
https://stackoverflow.com/questions/37000488/how-to-plot-multi-objectives-pareto-frontier-with-deap-in-python
'''

def simple_cull(inputPoints, dominates):
    paretoPoints = set()
    candidateRowNr = 0
    dominatedPoints = set()
    while True:
        candidateRow = inputPoints[candidateRowNr]
        inputPoints.remove(candidateRow)
        rowNr = 0
        nonDominated = True
        while len(inputPoints) != 0 and rowNr < len(inputPoints):
            row = inputPoints[rowNr]
            if dominates(candidateRow, row):
                # If it is worse on all features remove the row from the array
                inputPoints.remove(row)
                dominatedPoints.add(tuple(row))
            elif dominates(row, candidateRow):
                nonDominated = False
                dominatedPoints.add(tuple(candidateRow))
                rowNr += 1
            else:
                rowNr += 1

        if nonDominated:
            # add the non-dominated point to the Pareto frontier
            paretoPoints.add(tuple(candidateRow))

        if len(inputPoints) == 0:
            break
    return paretoPoints, dominatedPoints

def dominates(row, candidateRow):
    return sum([row[x] < candidateRow[x] for x in range(len(row))]) == len(row) 



'''
data extracted from GMapsSIGSPATIAL.ipynb (to avoid costly recomputation, since those cost actually real money...)
for demo purposes the both dictionaries (map_placeid_loc , resdict) will be forwarded and not explicitly hard-coded like here
'''

map_placeid_loc = {'ChIJ25acP_JVskcR66VeNl7Dki0': ['Steekberg 1, Kiel'],
 'ChIJ4-7amiFUskcRx-eGSnsU11k': ['Rendsburger Landstraße 240, Kiel'],
 'ChIJ464ABndWskcRPJCnE1Dik1g': ['Eichkamp 24, Kiel'],
 'ChIJARO96YFWskcR2Js37kmDSDk': ['Mühlendamm 1, Kiel'],
 'ChIJBTcEixBXskcRnRE7xmZfW_E': ['Villacher Str. 8, Kiel'],
 'ChIJDZ-XVWhWskcR2dwtbNtK8O8': ['Preußerstraße 1-9, Kiel'],
 'ChIJD_5lsftVskcRXxLeRcwxGE4': ['Torfmoorkamp 10, Kiel'],
 'ChIJGVJsHsn4skcRTahcUbZPD7g': ['Friedrichsruher Weg 4, Altenholz'],
 'ChIJLaOVCZlWskcRB0eVEk7UpsI': ['Hamburger Ch 180, Kiel'],
 'ChIJNwH-4NP-skcRvBKgDuhimL4': ['Erdbeerfeld 2, Altenholz'],
 'ChIJO09VeTUAs0cRkIe4HZECZiY': ['Ravensberg 4c, Gettorf'],
 'ChIJU2qUhBX4skcRDZJQoU4KCUQ': ['Tobringer 2, Heikendorf'],
 'ChIJX5MyNb5XskcRjdhvakFidJs': ['Schönkirchener Str. 80 b, Kiel'],
 'ChIJXSeN6KVXskcRFJGcvDemrG4': ['Philipp-Reis-Weg 4, Kiel'],
 'ChIJ_____1hWskcR1zT-wJoIK18': ['Augustenstraße 30a-32, Kiel'],
 'ChIJb9_69MVWskcRuM_DnXLLQ5c': ['Grot Steenbusch 35, Kiel'],
 'ChIJbdJh06xVskcRmTMLU8MGjxw': ['Kurt-Schumacher-Platz 4, Kiel'],
 'ChIJf27rluP4skcRDmkmbVOug6w': ['Friedrichsorter Str. 2, Kiel'],
 'ChIJr3BHgGxWskcRTMhzuUNuFCs': ['Droysenstraße 5, Kiel'],
 'ChIJs9hxzan4skcRqcd-I9tLT34': ['Prinz-Heinrich-Straße 20, Kiel'],
 'ChIJsQVmc-9WskcRS3BV6SyHPoc': ['Theodor-Heuss-Ring 136, Kiel'],
 'ChIJtRZoiv9WskcR7ohQ-eBS7T0': ['Ostring 232, Kiel'],
 'ChIJv1foXo1WskcRc46YWdst7G8': ['Stormarnstraße 29, Kiel'],
 'ChIJwQCXp-BZskcRfl5Hmm--c5c': ['Carl-Zeiss-Straße 2-10, Schwentinental'],
 'ChIJzZROoMdXskcRE_QE-o8f6b4': ['Langer Rehm 10, Kiel']}




resdict = {'ChIJ4-7amiFUskcRx-eGSnsU11k': [54.3002176, 10.0878471, 6.612, 14.85, 0.8627450980392157, ['Lidl', 'Rewe', 'EDEKA', 'REWE'], 5.750335481082384], 
           'ChIJDZ-XVWhWskcR2dwtbNtK8O8': [54.3293103, 10.1336699, 2.938, 7.316666666666666, 0.7945205479452054, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJsQVmc-9WskcRS3BV6SyHPoc': [54.3009059, 10.1422826, 6.389, 16.416666666666668, 1.94, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJs9hxzan4skcRqcd-I9tLT34': [54.36001830000001, 10.1336367, 5.267, 7.533333333333333, 0.703125, ['REWE'], 7.0189393939393945], 
           'ChIJv1foXo1WskcRc46YWdst7G8': [54.30565739999999, 10.126084, 4.824, 12.883333333333333, 0.7910447761194029, ['Rewe', 'EDEKA', 'Lidl', 'REWE'], 5.750335481082384], 
           'ChIJr3BHgGxWskcRTMhzuUNuFCs': [54.33495600000001, 10.1320073, 2.985, 7.733333333333333, 1.0158730158730158, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJbdJh06xVskcRmTMLU8MGjxw': [54.3237325, 10.054179, 5.291, 10.35, 0.8857142857142857, ['Rewe', 'Kaufland', 'REWE'], 6.183712121212121], 
           'ChIJzZROoMdXskcRE_QE-o8f6b4': [54.3345708, 10.1901217, 11.089, 23.466666666666665, 1.0, ['Lidl', 'EDEKA', 'REWE'], 6.366812753809658], 
           'ChIJXSeN6KVXskcRFJGcvDemrG4': [54.3175444, 10.1830397, 9.714, 22.466666666666665, 1.21875, ['Lidl', 'REWE'], 6.645164884135473], 
           'ChIJD_5lsftVskcRXxLeRcwxGE4': [54.3541912, 10.1050172, 2.393, 5.566666666666666, 0.8529411764705882, ['EDEKA', 'REWE'], 6.7405872636135795], 
           'ChIJO09VeTUAs0cRkIe4HZECZiY': [54.41428999999999, 9.9833521, 15.152, 12.983333333333333, 0.34328358208955223, ['Lidl', 'EDEKA', 'REWE'], 6.366812753809658], 
           'ChIJ464ABndWskcRPJCnE1Dik1g': [54.3303344, 10.1112091, 1.829, 5.316666666666666, 0.5555555555555556, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJLaOVCZlWskcRB0eVEk7UpsI': [54.2992614, 10.1049832, 6.036, 15.666666666666666, 0.8382352941176471, ['Rewe', 'EDEKA', 'Lidl', 'REWE'], 5.750335481082384], 
           'ChIJBTcEixBXskcRnRE7xmZfW_E': [54.2978171, 10.1737386, 8.644, 18.083333333333332, 0.35714285714285715, ['REWE', 'Lidl'], 6.645164884135473], 
           'ChIJ_____1hWskcR1zT-wJoIK18': [54.31439839999999, 10.1467048, 6.03, 18.333333333333332, 1.6842105263157894, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJb9_69MVWskcRuM_DnXLLQ5c': [54.2823898, 10.1321279, 8.27, 16.883333333333333, 0.5760869565217391, ['EDEKA'], 7.357064536340852], 
           'ChIJtRZoiv9WskcR7ohQ-eBS7T0': [54.3125073, 10.159109, 7.563, 18.266666666666666, 0.7076923076923077, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658], 
           'ChIJNwH-4NP-skcRvBKgDuhimL4': [54.4102781, 10.1301003, 10.924, 11.033333333333333, 0.5625, ['EDEKA'], 7.357064536340852], 
           'ChIJGVJsHsn4skcRTahcUbZPD7g': [54.3846564, 10.1388724, 8.173, 8.883333333333333, 0.2033898305084746, ['Rewe', 'EDEKA', 'REWE'], 6.124109990886307], 
           'ChIJU2qUhBX4skcRDZJQoU4KCUQ': [54.3766901, 10.221551, 17.605, 27.916666666666668, 0.49333333333333335, ['EDEKA', 'REWE'], 6.7405872636135795], 
           'ChIJX5MyNb5XskcRjdhvakFidJs': [54.3290837, 10.1984425, 10.968, 22.766666666666666, 1, ['Lidl', 'EDEKA', 'REWE'], 6.366812753809658], 
           'ChIJARO96YFWskcR2Js37kmDSDk': [54.3118413, 10.0985562, 4.303, 6.0, 0.4782608695652174, ['Rewe', 'EDEKA', 'Lidl', 'REWE'], 5.750335481082384], 
           'ChIJ25acP_JVskcR66VeNl7Dki0': [54.3532932, 10.0922142, 2.546, 6.133333333333334, 0.4, ['EDEKA', 'REWE'], 6.7405872636135795], 
           'ChIJf27rluP4skcRDmkmbVOug6w': [54.3960602, 10.1731945, 10.957, 13.7, 1.0377358490566038, ['Lidl', 'Rewe', 'REWE'], 6.0286876114082], 
           'ChIJwQCXp-BZskcRfl5Hmm--c5c': [54.28448460000001, 10.221673, 13.027, 21.683333333333334, 0.8484848484848485, ['Lidl', 'REWE'], 6.645164884135473]}



arr_locations = []
arr_distances = []
arr_durations = []
arr_utility = []
arr_poptimes = []

keyslist = resdict.keys()

for key in keyslist:
    lat, lng, dist, dur, pop, vicc, util = resdict[key]
    arr_locations.append(key)
    arr_distances.append(dist)
    arr_durations.append(dur)
    arr_utility.append(util)
    arr_poptimes.append(pop)


arr_labels = []
for e in arr_locations:
    address = map_placeid_loc[e]
    arr_labels.append(address)


inputPoints = list(zip(arr_distances, arr_durations, arr_utility))
inputVectArr = list(zip(arr_distances, arr_durations, arr_utility))

paretoPoints, dominatedPoints = simple_cull(inputPoints, dominates)

#populate dictionary with dominating objects with storelocation code as keys
fusiondict = {}
for dominate in paretoPoints:
    for i in range(len(keyslist)):
        if dominate == inputVectArr[i]:
            fusiondict[list(keyslist)[i]] = [dominate]
        
    

#now we need to determine by which feature/pair of features each pareto component dominates (min. values) 
#(distance, duration, utility etc.)
from operator import itemgetter


mindistvec = [min(paretoPoints,key=itemgetter(0)), 'Store with shortest distance: ']
mindurvec = [min(paretoPoints,key=itemgetter(1)), 'Store with shortest traffic duration: ']
minutilvec = [min(paretoPoints,key=itemgetter(2)), 'Store with best utility: ']    


#enrich fusiondict with min-solutions:
for key in fusiondict.keys():
    if fusiondict[key][0] == mindistvec[0]:
        fusiondict[key].append(mindistvec[1])
    elif fusiondict[key][0] == mindurvec[0]:
        fusiondict[key].append(mindurvec[1])
    elif fusiondict[key][0] == minutilvec[0]:
        fusiondict[key].append(minutilvec[1])
    else:
        fusiondict[key].append('Trade-off solution from the skyline: ')


        
'''
the following output can be shown in a nice listing (website or app)
here we have a rudimentary printout so far
'''        
print('>>>Recommended stores in your vicinity based on different criteria<<\n')
for key in fusiondict.keys():
    #print(fusiondict[key][1])
    print('Street: ',map_placeid_loc[key][0])
    print('Distance [km]: ',"%.2f" % resdict[key][2])
    print('Duration [min]: ',"%.2f" % resdict[key][3])
    print('Utility Score: ',"%.2f" % resdict[key][6] )
    print('Other stores in 2km airline: ',resdict[key][5])
    print('Popular times factor: ',"%.2f" % resdict[key][4])
    print('Link to Google Maps: ','https://www.google.com/maps/search/?api=1&query='+str(resdict[key][0])+','+str(resdict[key][1]))
    print('------------------------------------------------\n')




fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Distances [km]')
ax.set_ylabel('Traffic Durations [min]')
ax.set_zlabel('Utility')

volume = (5 * np.asarray(arr_poptimes))**4


dp = np.array(list(dominatedPoints))
pp = np.array(list(paretoPoints))

ax.scatter(dp[:,0],dp[:,1],dp[:,2],color='green')
ax.scatter(pp[:,0],pp[:,1],pp[:,2],color='red')


triang = mtri.Triangulation(pp[:,0],pp[:,1])
ax.plot_trisurf(triang,pp[:,2],color='red', alpha=0.5)


for i in range(len(arr_labels)): #plot each point + it's index as text above
    ax.scatter(arr_distances[i],arr_durations[i],arr_utility[i], s=volume[i], alpha=0.5) 
    ax.text(arr_distances[i],arr_durations[i],arr_utility[i],  '%s' % (arr_labels[i][0]), size=8, zorder=1,  color='k') 

plt.show()

>>>Recommended stores in your vicinity based on different criteria<<

Street:  Rendsburger Landstraße 240, Kiel
Distance [km]:  6.61
Duration [min]:  14.85
Utility Score:  5.75
Other stores in 2km airline:  ['Lidl', 'Rewe', 'EDEKA', 'REWE']
Popular times factor:  0.86
Link to Google Maps:  https://www.google.com/maps/search/?api=1&query=54.3002176,10.0878471
------------------------------------------------

Street:  Mühlendamm 1, Kiel
Distance [km]:  4.30
Duration [min]:  6.00
Utility Score:  5.75
Other stores in 2km airline:  ['Rewe', 'EDEKA', 'Lidl', 'REWE']
Popular times factor:  0.48
Link to Google Maps:  https://www.google.com/maps/search/?api=1&query=54.3118413,10.0985562
------------------------------------------------

Street:  Eichkamp 24, Kiel
Distance [km]:  1.83
Duration [min]:  5.32
Utility Score:  6.37
Other stores in 2km airline:  ['EDEKA', 'Lidl', 'REWE']
Popular times factor:  0.56
Link to Google Maps:  https://www.google.com/maps/search/?api=1&query=54.3303344,10.1112